In [1]:
# Import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import google_key

In [2]:
#import CSV from WeatherPy as DataFrame
city_data_file = "../WeatherPy/cities.csv"

#read into df
city_data_df = pd.read_csv(city_data_file)

#get rid of index column that was read in
city_data_df = city_data_df[["City", "Lat", "Lng", "Max Temp", "Cloudiness", "Humidity", "Wind Speed",
                            "Country", "Date"]]
city_data_df.head()

,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
0,hilo,19.7297,-155.0900,73.40,90,83,6.26,US,1611782150
1,albany,42.6001,-73.9662,30.00,81,62,13.00,US,1611782082
2,bredasdorp,-34.5322,20.0403,57.20,96,72,5.75,ZA,1611782144
3,cockburn town,21.4612,-71.1419,80.60,40,54,10.36,TC,1611782640
4,atuona,-9.8000,-139.0333,78.48,2,78,15.17,PF,1611782151


In [3]:
#double check the data types of all numbers
city_data_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Cloudiness      int64
Humidity        int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [33]:
##Humidity Heatmap
#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.
#Add Heatmap layer to map.

#store city lat/lng and humidity into variables to use for heat map
locations = city_data_df[["Lat", "Lng"]]

#convert humidity from int to float
humidity = city_data_df["Humidity"].astype(float)

dtype('float64')

In [34]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
## Create New DF to fit weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

# Narrow down DF to find cities with ideal weather conditions
# My ideal weather conditions: Max temp > 65 but less than 85, wind speed < 5 mph, less than 20% cloudiness
hotel_df = city_data_df.loc[(city_data_df["Max Temp"] > 65) & (city_data_df["Max Temp"] < 85) 
                                    & (city_data_df["Wind Speed"] < 5) & (city_data_df["Cloudiness"] < 20)]
hotel_df

,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date
46,coolum beach,-26.5333,153.1000,80.01,11,82,3.00,AU,1611782647
54,cururupu,-1.8283,-44.8683,80.78,10,76,4.14,BR,1611782648
75,odweyne,9.4092,45.0640,65.75,0,41,4.45,SO,1611782651
86,oussouye,12.4850,-16.5469,77.00,0,88,2.30,SN,1611782413
89,hervey bay,-25.2986,152.8535,79.00,7,79,4.00,AU,1611782584
104,tiznit,29.5833,-9.5000,65.46,0,42,1.28,MA,1611782188
108,saint-philippe,-21.3585,55.7679,78.80,0,69,4.61,RE,1611782140
129,kabala,9.5833,-11.5500,75.96,0,19,2.21,SL,1611782660
155,odienne,9.5100,-7.5692,77.41,0,18,3.15,CI,1611782663
171,yanam,16.7333,82.2167,68.99,0,90,2.17,IN,1611782665


In [ ]:
## Hotel Map
# Store into variable named hotel_df.
# Add a "Hotel Name" column to the DataFrame.
# Set parameters to search for hotels with 5000 meters.
# Hit the Google Places API for each city's coordinates.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

In [31]:
#create blank column in data frame called Hotel Name
hotel_df["Hotel Name"] = ""

# create a params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "hotel",
    "key": google_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging", keyword: "hotel", and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    print(f"Retrieving Results for Index {index}.")
    # make request and print url
    hotel_address = requests.get(base_url, params=params)
    
    #print hotel address url in order to check that it was called correctly
    #print(hotel_address.url)

    # convert to json
    hotel_address = hotel_address.json()
    
    #printed json to check syntax/where the name of the hotels is located
    #print(json.dumps(hotel_address, indent=4, sort_keys=True))

    # Incorporate a try-except to skip any cities that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

C:\Users\erinc\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\erinc\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Retrieving Results for Index 46.
Retrieving Results for Index 54.
Retrieving Results for Index 75.
Retrieving Results for Index 86.
Retrieving Results for Index 89.
Retrieving Results for Index 104.
Retrieving Results for Index 108.
Retrieving Results for Index 129.
Retrieving Results for Index 155.
Retrieving Results for Index 171.
Retrieving Results for Index 175.
Retrieving Results for Index 197.
Retrieving Results for Index 231.
Retrieving Results for Index 280.
Retrieving Results for Index 323.
Retrieving Results for Index 368.
Retrieving Results for Index 426.
Retrieving Results for Index 439.
Retrieving Results for Index 450.
Retrieving Results for Index 472.
Retrieving Results for Index 526.


,City,Lat,Lng,Max Temp,Cloudiness,Humidity,Wind Speed,Country,Date,Hotel Name
46,coolum beach,-26.5333,153.1000,80.01,11,82,3.00,AU,1611782647,On The Beach Noosa Resort
54,cururupu,-1.8283,-44.8683,80.78,10,76,4.14,BR,1611782648,Pousada Praia Bonita
75,odweyne,9.4092,45.0640,65.75,0,41,4.45,SO,1611782651,Al shukri hotel
86,oussouye,12.4850,-16.5469,77.00,0,88,2.30,SN,1611782413,Club Med Cap Skirring - Senegal
89,hervey bay,-25.2986,152.8535,79.00,7,79,4.00,AU,1611782584,Mantra Hervey Bay
104,tiznit,29.5833,-9.5000,65.46,0,42,1.28,MA,1611782188,Ajgal Maison d'hôtes / Ajgal Guest House
108,saint-philippe,-21.3585,55.7679,78.80,0,69,4.61,RE,1611782140,LODGE PALMAE
129,kabala,9.5833,-11.5500,75.96,0,19,2.21,SL,1611782660,Stando Hotel
155,odienne,9.5100,-7.5692,77.41,0,18,3.15,CI,1611782663,HÔTEL SARAH
171,yanam,16.7333,82.2167,68.99,0,90,2.17,IN,1611782665,RVR Sarovar Portico Dindi


In [35]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(location, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)

#display
fig

In [ ]:
# Add marker hotel layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig


